In [14]:
import rasterio
import numpy as np
import pickle
import pandas as pd
from pyproj import Transformer, Geod

transformer_ps2latlon = Transformer.from_crs("EPSG:3031", "EPSG:4326")
transformer_latlon2ps = Transformer.from_crs("EPSG:4326", "EPSG:3031")
geodesic = Geod(ellps='WGS84')

# Load velocity data

In [6]:
vx_in = "/home/ram21/notebooks/brunt-is2-rifts/halloween/landsat/ANT_G0240_2015_vx.tif"
vy_in = "/home/ram21/notebooks/brunt-is2-rifts/halloween/landsat/ANT_G0240_2015_vy.tif"

vx15img = rasterio.open(vx_in)
vy15img = rasterio.open(vy_in)

vx15imgar = vx15img.read(1)
vy15imgar = vy15img.read(1)

vx15imgar[vx15imgar==-32767]=np.nan
vy15imgar[vy15imgar==-32767]=np.nan

vv15imgar = np.sqrt((vx15imgar**2) + (vy15imgar**2))

# Load rift geometry

In [24]:
rift_path = pickle.load( open( '/data/fast1/arc/feature_outlines/halloween_path.pickle', "rb" ) )
riftx,rifty = np.array(rift_path.xy)

<ipython-input-24-13631fc031d2>:1: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  rift_path = pickle.load( open( '/data/fast1/arc/feature_outlines/halloween_path.pickle', "rb" ) )


# Load GPS

In [15]:
# western GNSS pair
gps = pd.read_csv('/home/ram21/notebooks/brunt-is2-rifts/GPS/bas_marsh_monthly_halloween_updated.csv')

hh00_x = np.array(gps.PSX[ gps.Site=='HH00' ])
hh00_y = np.array(gps.PSY[ gps.Site=='HH00' ])
tt05_x = np.array(gps.PSX[ gps.Site=='TT05' ])
tt05_y = np.array(gps.PSY[ gps.Site=='TT05' ])

hh00_yyyy = np.array(gps.YYYY[gps.Site=='HH00'])
hh00_mm   = np.array(gps.MM[gps.Site=='HH00'])
tt05_yyyy = np.array(gps.YYYY[gps.Site=='TT05'])
tt05_mm   = np.array(gps.MM[gps.Site=='TT05'])

hh00_t = np.array(gps.YYYY[gps.Site=='HH00' ]) + (np.array(gps.MM[gps.Site=='HH00' ]) / 12) - (15/365)
tt05_t = np.array(gps.YYYY[gps.Site=='TT05' ]) + (np.array(gps.MM[gps.Site=='TT05' ]) / 12) - (15/365)


tt05_x = np.delete(tt05_x,[30,31,36,37,38,39,40,41,42,43,44,45,46])
tt05_y = np.delete(tt05_y,[30,31,36,37,38,39,40,41,42,43,44,45,46])
tt05_yyyy = np.delete(tt05_yyyy,[30,31,36,37,38,39,40,41,42,43,44,45,46])
tt05_mm = np.delete(tt05_mm,[30,31,36,37,38,39,40,41,42,43,44,45,46])
tt05_t = np.delete(tt05_t,[30,31,36,37,38,39,40,41,42,43,44,45,46])

# calculate the separation distance of GNSS receiver pair
hh00_tt05_sep_d = np.sqrt((hh00_x - tt05_x)**2 + (hh00_y - tt05_y)**2)

# rift segment 4
# calculate angle of this segment of rift
[latr4,longr4] = transformer_ps2latlon.transform(riftx[4],rifty[4])
[latr5,longr5] = transformer_ps2latlon.transform(riftx[5],rifty[5])
fwd_az_rift,back_az_rift,dist_rift = geodesic.inv(longr4,latr4,longr5,latr5)

fwd_azimuth_gps  = []
fwd_azimuth_rift = []
gps_rift_diff = []

# for each month, calculate GNSS baseline angle
# and angular offset to rift
for gpsi in range(0,len(hh00_x)):
    [lathh,longhh] = transformer_ps2latlon.transform(hh00_x[gpsi],hh00_y[gpsi])
    [lattt,longtt] = transformer_ps2latlon.transform(tt05_x[gpsi],tt05_y[gpsi])
    fwd_az_gps,back_az_gps,dist_gps = geodesic.inv(longtt,lattt,longhh,lathh)
    fwd_azimuth_gps.append(fwd_az_gps)
    fwd_azimuth_rift.append(fwd_az_rift)
    gps_rift_diff.append(fwd_az_rift-fwd_az_gps)

# calculate the component of separation distance of the GNSS receiver pair perpendicular to rift
actual_hh00_tt05_sep_d = np.abs(np.sin(np.deg2rad(gps_rift_diff))) * hh00_tt05_sep_d
    

<ipython-input-15-73f3dd3510fa>:1: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  rift_path = pickle.load( open( '/data/fast1/arc/feature_outlines/halloween_path.pickle', "rb" ) )


In [17]:
# eastern GNSS pair
ss00_x = np.array(gps.PSX[ gps.Site=='SS00' ])
ss00_y = np.array(gps.PSY[ gps.Site=='SS00' ])
tt04_x = np.array(gps.PSX[ gps.Site=='TT04' ])
tt04_y = np.array(gps.PSY[ gps.Site=='TT04' ])

ss00_yyyy = np.array(gps.YYYY[gps.Site=='SS00'])
ss00_mm   = np.array(gps.MM[gps.Site=='SS00'])
tt04_yyyy = np.array(gps.YYYY[gps.Site=='TT04'])
tt04_mm   = np.array(gps.MM[gps.Site=='TT04'])

ss00_t = np.array(gps.YYYY[gps.Site=='SS00' ]) + (np.array(gps.MM[gps.Site=='SS00' ]) / 12) - (15/365)
tt04_t = np.array(gps.YYYY[gps.Site=='TT04' ]) + (np.array(gps.MM[gps.Site=='TT04' ]) / 12) - (15/365)

ss00_x = np.delete(ss00_x,[3,4,5])
ss00_y = np.delete(ss00_y,[3,4,5])
ss00_yyyy = np.delete(ss00_yyyy,[3,4,5])
ss00_mm = np.delete(ss00_mm,[3,4,5])
ss00_t = np.delete(ss00_t,[3,4,5])

tt04_x = np.delete(tt04_x,[0,4,5])
tt04_y = np.delete(tt04_y,[0,4,5])
tt04_yyyy = np.delete(tt04_yyyy,[0,4,5])
tt04_mm = np.delete(tt04_mm,[0,4,5])
tt04_t = np.delete(tt04_t,[0,4,5])

# calculate the separation distance of GNSS receiver pair
ss00_tt04_sep_d = np.sqrt((ss00_x - tt04_x)**2 + (ss00_y - tt04_y)**2)

# rift segment 8
# calculate angle of this segment of rift
[latr8,longr8] = transformer_ps2latlon.transform(riftx[8],rifty[8])
[latr9,longr9] = transformer_ps2latlon.transform(riftx[9],rifty[9])
fwd_az_rift,back_az_rift,dist_rift = geodesic.inv(longr8,latr8,longr9,latr9)

fwd_azimuth_gps  = []
fwd_azimuth_rift = []
gps_rift_diff = []

# for each month, calculate GNSS baseline angle
# and angular offset to rift
for gpsj in range(0,len(ss00_x)):
    [lathh,longhh] = transformer_ps2latlon.transform(ss00_x[gpsj],ss00_y[gpsj])
    [lattt,longtt] = transformer_ps2latlon.transform(tt04_x[gpsj],tt04_y[gpsj])
    fwd_az_gps,back_az_gps,dist_gps = geodesic.inv(longtt,lattt,longhh,lathh)
    fwd_azimuth_gps.append(fwd_az_gps)
    fwd_azimuth_rift.append(fwd_az_rift)
    gps_rift_diff.append(fwd_az_rift-fwd_az_gps)

# calculate the component of separation distance of the GNSS receiver pair perpendicular to rift    
actual_ss00_tt04_sep_d = np.abs(np.sin(np.deg2rad(gps_rift_diff))) * ss00_tt04_sep_d


In [19]:
# extraction locations (mean location of GNSS)
hh00_x_mean = np.mean(hh00_x[0:30])
hh00_y_mean = np.mean(hh00_y[0:30])
tt05_x_mean = np.mean(tt05_x[0:30])
tt05_y_mean = np.mean(tt05_y[0:30])
ss00_x_mean = np.mean(ss00_x)
ss00_y_mean = np.mean(ss00_y)
tt04_x_mean = np.mean(tt04_x)
tt04_y_mean = np.mean(tt04_y)

print(hh00_x_mean,hh00_y_mean)

-696325.6 1438278.8666666667


In [20]:
for val in vx15img.sample([(hh00_x_mean, hh00_y_mean)]):
    print(val)
hh00_vx=val[0]
for val in vy15img.sample([(hh00_x_mean, hh00_y_mean)]): 
    print(val)
hh00_vy=val[0]    

for val in vx15img.sample([(tt05_x_mean, tt05_y_mean)]):
    print(val)
tt05_vx=val[0]    
for val in vy15img.sample([(tt05_x_mean, tt05_y_mean)]):
    print(val)
tt05_vy=val[0]

for val in vx15img.sample([(ss00_x_mean, ss00_y_mean)]):
    print(val)
ss00_vx=val[0]
for val in vy15img.sample([(ss00_x_mean, ss00_y_mean)]):
    print(val)
ss00_vy=val[0]

for val in vx15img.sample([(tt04_x_mean, tt04_y_mean)]):
    print(val)
tt04_vx=val[0]
for val in vy15img.sample([(tt04_x_mean, tt04_y_mean)]):
    print(val)
tt04_vy=val[0]
# print("hh00")
# print(hh00_vx,hh00_vy)
# print("tt05")
# print(tt05_vx,tt05_vy)
# print("ss00")
# print(ss00_vx,ss00_vy)
# print("tt04")
# print(tt04_vx,tt04_vy)


[-703.26044]
[-100.26767]
[-555.5864]
[-59.93567]
[-734.9666]
[14.168331]
[-600.4135]
[35.792732]
hh00
-703.26044 -100.26767
tt05
-555.5864 -59.93567
ss00
-734.9666 14.168331
tt04
-600.4135 35.792732


In [21]:
# Assumed location of GNSS receivers after 1 year (mean location + 1 year of ice flow)
hh00_x_yr = hh00_x_mean+(1*hh00_vx)
hh00_y_yr = hh00_y_mean+(1*hh00_vy)
tt05_x_yr = tt05_x_mean+(1*tt05_vx)
tt05_y_yr = tt05_y_mean+(1*tt05_vy)
ss00_x_yr = ss00_x_mean+(1*ss00_vx)
ss00_y_yr = ss00_y_mean+(1*ss00_vy)
tt04_x_yr = tt04_x_mean+(1*tt04_vx)
tt04_y_yr = tt04_y_mean+(1*tt04_vy)

# Calculate the flow velocity of the receivers
# and the flow angle of the receivers
# hh00
hh00_v = np.sqrt(hh00_vx**2 + hh00_vy**2)
[hh00_lat,hh00_long] = transformer_ps2latlon.transform(hh00_x_mean,hh00_y_mean)
[hh00_yr_lat,hh00_yr_long] = transformer_ps2latlon.transform(hh00_x_yr,hh00_y_yr)
hh00_fwd_azimuth,hh00_back_azimuth,hh00_distance = geodesic.inv(hh00_long, hh00_lat, hh00_yr_long, hh00_yr_lat)

# tt05
tt05_v = np.sqrt(tt05_vx**2 + tt05_vy**2)
[tt05_lat,tt05_long] = transformer_ps2latlon.transform(tt05_x_mean,tt05_y_mean)
[tt05_yr_lat,tt05_yr_long] = transformer_ps2latlon.transform(tt05_x_yr,tt05_y_yr)
tt05_fwd_azimuth,tt05_back_azimuth,tt05_distance = geodesic.inv(tt05_long, tt05_lat, tt05_yr_long, tt05_yr_lat)

# ss00
ss00_v = np.sqrt(ss00_vx**2 + ss00_vy**2)
[ss00_lat,ss00_long] = transformer_ps2latlon.transform(ss00_x_mean,ss00_y_mean)
[ss00_yr_lat,ss00_yr_long] = transformer_ps2latlon.transform(ss00_x_yr,ss00_y_yr)
ss00_fwd_azimuth,ss00_back_azimuth,ss00_distance = geodesic.inv(ss00_long, ss00_lat, ss00_yr_long, ss00_yr_lat)

# tt04
tt04_v = np.sqrt(tt04_vx**2 + tt04_vy**2)
[tt04_lat,tt04_long] = transformer_ps2latlon.transform(tt04_x_mean,tt04_y_mean)
[tt04_yr_lat,tt04_yr_long] = transformer_ps2latlon.transform(tt04_x_yr,tt04_y_yr)
tt04_fwd_azimuth,tt04_back_azimuth,tt04_distance = geodesic.inv(tt04_long, tt04_lat, tt04_yr_long, tt04_yr_lat)

print("Featured tracked velocity at mean GNSS receiver locations")
print("hh00 speed "+str(round(hh00_v,2))+" m/yr, angle "+str(round(hh00_fwd_azimuth,2))+" deg")
print("tt05 speed "+str(round(tt05_v,2))+" m/yr, angle "+str(round(tt05_fwd_azimuth,2))+" deg")
print("ss00 speed "+str(round(ss00_v,2))+" m/yr, angle "+str(round(ss00_fwd_azimuth,2))+" deg")
print("tt04 speed "+str(round(tt04_v,2))+" m/yr, angle "+str(round(tt04_fwd_azimuth,2))+" deg")
print("")

# hh00 tt05 rift segment 4-5
[r4_lat,r4_long] = transformer_ps2latlon.transform(riftx[4],rifty[4])
[r5_lat,r5_long] = transformer_ps2latlon.transform(riftx[5],rifty[5])
r45_fwd_azimuth,r45_back_azimuth,r45_distance = geodesic.inv(r4_long, r4_lat, r5_long, r5_lat)

print("rift segment 4 angle "+str(round(r45_fwd_azimuth,2))+" deg")

# ss00 tt04 rift segment 8-9
[r8_lat,r8_long] = transformer_ps2latlon.transform(riftx[8],rifty[8])
[r9_lat,r9_long] = transformer_ps2latlon.transform(riftx[9],rifty[9])
r89_fwd_azimuth,r89_back_azimuth,r89_distance = geodesic.inv(r8_long, r8_lat, r9_long, r9_lat)

print("rift segment 8 angle "+str(round(r89_fwd_azimuth,2))+" deg")
print("")

# rift perpendicular components of velocity
hh00_v_rperp = np.abs(np.sin(np.deg2rad(r45_fwd_azimuth - hh00_fwd_azimuth))) * hh00_v
tt05_v_rperp = np.abs(np.sin(np.deg2rad(r45_fwd_azimuth - tt05_fwd_azimuth))) * tt05_v
hh00_tt05_vdiff_rperp = hh00_v_rperp - tt05_v_rperp

print("rift perpendicular component of velocity")
print("hh00 rift perp. "+str(round(hh00_v_rperp,2))+" m/yr")
print("tt05 rift perp. "+str(round(tt05_v_rperp,2))+" m/yr")
print("hh00 tt05 difference: "+str(round(hh00_tt05_vdiff_rperp,2))+" m/yr")

ss00_v_rperp = np.abs(np.sin(np.deg2rad(r89_fwd_azimuth - ss00_fwd_azimuth))) * ss00_v
tt04_v_rperp = np.abs(np.sin(np.deg2rad(r89_fwd_azimuth - tt04_fwd_azimuth))) * tt04_v
ss00_tt04_vdiff_rperp = ss00_v_rperp - tt04_v_rperp

print("ss00 rift perp. "+str(round(ss00_v_rperp,2))+" m/yr")
print("tt04 rift perp. "+str(round(tt04_v_rperp,2))+" m/yr")
print("ss00 tt04 difference: "+str(round(ss00_tt04_vdiff_rperp,2))+" m/yr")
print("")

# x and y components of perpendicular velocity (for figure)
w_rperp_angle = np.arctan((rifty[5]-rifty[4])/(riftx[5]-riftx[4]))
hh00_v_rperp_y = hh00_v_rperp*np.cos(w_rperp_angle)
hh00_v_rperp_x = -hh00_v_rperp*np.sin(w_rperp_angle)
tt05_v_rperp_y = tt05_v_rperp*np.cos(w_rperp_angle)
tt05_v_rperp_x = -tt05_v_rperp*np.sin(w_rperp_angle)

e_rperp_angle = np.arctan((rifty[9]-rifty[8])/(riftx[9]-riftx[8]))
ss00_v_rperp_y = ss00_v_rperp*np.cos(e_rperp_angle)
ss00_v_rperp_x = -ss00_v_rperp*np.sin(e_rperp_angle)
tt04_v_rperp_y = tt04_v_rperp*np.cos(e_rperp_angle)
tt04_v_rperp_x = -tt04_v_rperp*np.sin(e_rperp_angle)


Featured tracked velocity at mean GNSS receiver locations
hh00 speed 710.37 m/yr, angle -72.28 deg
tt05 speed 558.81 m/yr, angle -70.48 deg
ss00 speed 735.1 m/yr, angle -63.72 deg
tt04 speed 601.48 m/yr, angle -61.39 deg

rift segment 4 angle 81.58 deg
rift segment 8 angle 77.92 deg

rift perpendicular component of velocity
hh00 rift perp. 313.01 m/yr
tt05 rift perp. 261.9 m/yr
hh00 tt05 difference: 51.11 m/yr
ss00 rift perp. 456.23 m/yr
tt04 rift perp. 392.16 m/yr
ss00 tt04 difference: 64.08 m/yr



## Correct GPS for divergence

In [22]:
#hh00_t,actual_hh00_tt05_sep_d
hh00_t_diff = hh00_t - hh00_t[0]
div_cor_hh = hh00_tt05_vdiff_rperp * hh00_t_diff
actual_hh00_tt05_sep_d_div = actual_hh00_tt05_sep_d - div_cor_hh
#print(actual_hh00_tt05_sep_d_div)

#ss00_t,actual_ss00_tt04_sep_d
ss00_t_diff = ss00_t - ss00_t[0]
div_cor_ss = ss00_tt04_vdiff_rperp * ss00_t_diff
actual_ss00_tt04_sep_d_div = actual_ss00_tt04_sep_d - div_cor_ss
#print(actual_hh00_tt05_sep_d_div)

## Save the output

In [34]:
variables = {
    'hh00_x_mean': hh00_x_mean,
    'tt05_x_mean': tt05_x_mean,
    'ss00_x_mean': ss00_x_mean,
    'tt04_x_mean': tt04_x_mean,
    'hh00_y_mean': hh00_y_mean,
    'tt05_y_mean': tt05_y_mean,
    'ss00_y_mean': ss00_y_mean,
    'tt04_y_mean': tt04_y_mean,
    'hh00_vx': hh00_vx,
    'tt05_vx': tt05_vx,
    'ss00_vx': ss00_vx,
    'tt04_vx': tt04_vx,
    'hh00_vy': hh00_vy,
    'tt05_vy': tt05_vy,
    'ss00_vy': ss00_vy,
    'tt04_vy': tt04_vy,
    'hh00_v_rperp_x': hh00_v_rperp_x,
    'tt05_v_rperp_x': tt05_v_rperp_x,
    'ss00_v_rperp_x': ss00_v_rperp_x,
    'tt04_v_rperp_x': tt04_v_rperp_x,
    'hh00_v_rperp_y': hh00_v_rperp_y,
    'tt05_v_rperp_y': tt05_v_rperp_y,
    'ss00_v_rperp_y': ss00_v_rperp_y,
    'tt04_v_rperp_y': tt04_v_rperp_y,
    'hh00_t': hh00_t,
    'hh00_x': hh00_x,
    'hh00_y': hh00_y,
    'actual_hh00_tt05_sep_d_div': actual_hh00_tt05_sep_d_div,
    'tt04_x':tt04_x,
    'tt04_y':tt04_y,
    'tt05_x':tt05_x,
    'tt05_y':tt05_y,
    'ss00_t':ss00_t,
    'ss00_x':ss00_x,
    'ss00_y':ss00_y,
    'actual_ss00_tt04_sep_d_div':actual_ss00_tt04_sep_d_div,
    'actual_hh00_tt05_sep_d':actual_hh00_tt05_sep_d,
    'actual_ss00_tt04_sep_d':actual_ss00_tt04_sep_d
}

# Save the variables into a pickle file
with open('data/processed-gps.pkl', 'wb') as f:
    pickle.dump(variables, f)

print("Variables have been saved successfully.")

Variables have been saved successfully.
